# Custom Demo: Ride Hire Example With Unit Testing

A custom demo based on Snowflake and Stream Ingest API

## Notebook Setup
Configure Notebook Snowflake connection

In [ ]:
from dotenv import load_dotenv

# Load the environment variables from the .env file (requires python-dotenv)
load_dotenv()

In [ ]:
# Import Tecton and other libraries
import logging
import os
import snowflake.connector
import tecton

logging.getLogger('snowflake.connector').setLevel(logging.WARNING)
logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

connection_parameters = {
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_PASSWORD'],
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "role": os.environ['SNOWFLAKE_ROLE'],
    "warehouse": os.environ['SNOWFLAKE_WAREHOUSE'],
    "database": os.environ['SNOWFLAKE_DATABASE'],
    "schema": os.environ['SNOWFLAKE_SCHEMA']
}

conn = snowflake.connector.connect(**connection_parameters)
tecton.snowflake_context.set_connection(conn)
tecton.conf.set("TECTON_OFFLINE_RETRIEVAL_COMPUTE_MODE", "rift")

# Quick helper function to query snowflake from a notebook
def query_snowflake(query):
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

tecton.version.summary()
query_snowflake("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_WAREHOUSE(), CURRENT_DATABASE()")

## Import the Tecton SDK into your notebook

You can use the Tecton SDK to interact with Tecton, develop, test and deploy new features, as well as generating offline training data and testing the online path.



In [ ]:
import tecton
tecton.version.summary()
tecton.who_am_i()
tecton.set_validation_mode("auto")

ws = tecton.get_workspace('jon-unit-testing')

## Data Source Candidates

Example data source(s) to be used for testing.

In [ ]:
from datetime import datetime, timedelta, timezone

start = datetime(2024, 4, 1, 8)
end = datetime(2024, 4, 4, 10, 0, 1)

orders_snowflake_batch_source = ws.get_data_source('orders_snowflake_batch_source')
orders_snowflake_batch_source.get_dataframe(start_time=start, end_time=end).to_pandas()

## Feature View Candidates

Example Feature View(s) to be used for testing.

### Batch Feature View Example

In [ ]:
import pandas as pd
input_pandas_df = pd.DataFrame(
    {
        "ORDER_CREATED": [datetime(2024, 5, 10), datetime(2024, 5, 11), datetime(2024, 5, 12)],
        "ORDER_ID": ["ORD-001", "ORD-002", "ORD-003"],
        "REQUESTER_ID": ["REQ-001", "REQ-001", "REQ-002"],
        "PAYMENT_AMOUNT": [42.42, 55.42, 100.00],
    }
)
input_pandas_df

In [ ]:
events_df = pd.DataFrame(
    {
        "REQUESTER_ID": ["REQ-001", "REQ-001", "REQ-002"],
        "ORDER_CREATED": [datetime(2024, 5, 12), datetime(2024, 5, 13), datetime(2024, 5, 14)],
    }
)
events_df

In [ ]:
requester_order_batch_metrics = ws.get_feature_view('requester_orders_batch_metrics')
output_df = requester_order_batch_metrics.get_features_for_events(
        events_df, mock_inputs={"orders": input_pandas_df}, from_source=True
).to_pandas()
output_df

In [ ]:
expected_df = pd.DataFrame({
    'REQUESTER_ID': ['REQ-001', 'REQ-001', 'REQ-002'],
    'ORDER_CREATED': ['2024-05-12', '2024-05-13', '2024-05-14'],
    'requester_order_batch_metrics__ORDER_ID_count_30d_1d': [2, 2, 1],
    'requester_order_batch_metrics__PAYMENT_AMOUNT_mean_30d_1d': [48.92, 48.92, 100.00]
})
expected_df

### On-Demand Feature View Example

In [ ]:
input_dict = {
    'orders_stream_metrics': {
        'PICKUP_LATITUDE_last_30d_continuous': 34.0522,
        'PICKUP_LONGITUDE_last_30d_continuous': -118.2437,
        'DROPOFF_LATITUDE_last_30d_continuous': 36.7783,
        'DROPOFF_LONGITUDE_last_30d_continuous': -119.4179
    }
}
input_dict

In [ ]:
calculate_distance = requester_order_batch_metrics = ws.get_feature_view('calculate_distance')
output_df = calculate_distance.run_transformation(input_data=input_dict)
output_df


## Unit Testing

Tecton supports running local unit tests. Unit tests can be defined in feature repositories in file paths matching the pattern **/tests/*.py.

Tests can be run when the following commands are executed:

- tecton apply: Runs the tests and applies the repo if the tests pass.

- tecton plan: Runs the tests and shows the changes that would be made to the repo if the changes were applied.

- tecton test: Runs the tests, only.

Please refer to the following for examples:

- data_sources/tests/test_orders_snowflake_batch_source.py

- feature_views/tests/test_requester_orders_batch_metrics.py

- feature_views/tests/test_calculate_distance.py